In [3]:
import torch
from src.gaussian_diffusion import GaussianDiffusion
from src.Unet import ConditionalDiffusionUNet
from src.classifier import UNetClassifier

In [4]:
# 参数配置
num_classes = 7  # 根据实际类别数修改
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 加载模型 -------------------------------------------------
unet = ConditionalDiffusionUNet(num_classes=num_classes, time_dim=128, label_dim=64).to(device)
unet.load_state_dict(torch.load("best_unet.pth", map_location=device))

classifier = UNetClassifier(num_classes).to(device)
classifier.load_state_dict(torch.load("best_noisy_classifier.pth", map_location=device))
classifier.eval()

# 初始化扩散模型 --------------------------------------------
diffusion = GaussianDiffusion(
    num_steps=64,
    model=unet,
    classifier=classifier,
    classifier_scale=1.0  # 调节引导强度
).to(device)

# 生成配置 -------------------------------------------------
batch_size = 1
target_labels = torch.tensor([0], device=device)  # 手动指定目标类别
assert len(target_labels) == batch_size, "标签数量需与批次大小一致"

# 执行生成 -------------------------------------------------
generated = diffusion.p_sample_loop(
    shape=(batch_size, 1, 24, 50),
    batch_labels=target_labels,
    noise=None,  # 可传入自定义噪声
    clip_denoised=True,
    progress=True
)

# 结果验证 -------------------------------------------------
print("生成样本范围:", generated.min().item(), "~", generated.max().item())  # 应在[-1,1]之间

# 分类验证（可选）------------------------------------------
with torch.no_grad():
    logits = classifier(generated, torch.zeros(batch_size, device=device).long())
    preds = logits.argmax(dim=1)
    accuracy = (preds == target_labels).float().mean()
    print(f"生成样本分类准确率: {accuracy.item():.2%}")


FileNotFoundError: [Errno 2] No such file or directory: 'best_unet.pth'